In [1]:
import numpy as np
import open3d as o3d
import imageio.v3

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def _copy_mesh(mesh: o3d.geometry.TriangleMesh) -> o3d.geometry.TriangleMesh:
    
    m = o3d.geometry.TriangleMesh(mesh)
    m.vertices = o3d.utility.Vector3dVector(np.asarray(mesh.vertices).copy())
    m.triangles = o3d.utility.Vector3iVector(np.asarray(mesh.triangles).copy())
    if mesh.has_vertex_normals():
        m.vertex_normals = o3d.utility.Vector3dVector(np.asarray(mesh.vertex_normals).copy())
    if mesh.has_triangle_normals():
        m.triangle_normals = o3d.utility.Vector3dVector(np.asarray(mesh.triangle_normals).copy())
    return m

In [3]:
def generate_gif(meshes, colors, out_path):
    n_frames = 60
    width, height = 800, 600

    #mesh.compute_vertex_normals()
    #mesh.paint_uniform_color([0.949, 0.816, 0.455])
    #spheres = landmarks_as_spheres(landmarks.points, [1,0,0], radius=0.5)
    
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=True, width=width, height=height)
    #vis.add_geometry(mesh)
    for idx, geom in enumerate(meshes):
        if isinstance(geom, o3d.geometry.TriangleMesh):
            geom = _copy_mesh(geom)
            geom.compute_vertex_normals()
            geom.paint_uniform_color(colors[idx])
            vis.add_geometry(geom)

        elif isinstance(geom, o3d.geometry.PointCloud):
            geom = o3d.geometry.PointCloud(geom)  # copy
            geom.paint_uniform_color(colors[idx])
            vis.add_geometry(geom)
        else:
            raise TypeError(f"Unsupported geometry type: {type(geom)}")

    opt = vis.get_render_option()
    #opt.light_on = True
    opt.background_color = np.array([5.1, 6.7, 9])
    
    ctr = vis.get_view_control()
    bbox = geom.get_axis_aligned_bounding_box()
    center = bbox.get_center()
    
    radius = np.linalg.norm(bbox.get_extent()) * 1.5
    
    frames = []
    for i in range(n_frames):
        angle = 2 * np.pi * i / n_frames
    
        # --- CAMERA PATH (EDIT HERE) ---
        # orbit around z-axis with slight elevation
        cam_x = center[0] + radius * np.cos(angle)
        cam_y = center[1] + radius * np.sin(angle)
        cam_z = center[2] + radius * 0.3
    
        ctr.set_lookat(center)
        ctr.set_front([center[0] - cam_x,
                       center[1] - cam_y,
                       center[2] - cam_z])
        ctr.set_up([0, 0, 1])      # change if you want different "up"
        ctr.set_zoom(0.8)          # EDIT: closer/further
    
        vis.poll_events()
        vis.update_renderer()
    
        img = vis.capture_screen_float_buffer(False)
        img = (np.asarray(img) * 255).astype(np.uint8)
        frames.append(img)
    
    vis.destroy_window()
    # save GIF
    imageio.mimsave(out_path, frames, fps=10, loop=0)
    
    print(f"Saved rotating GIF as: {out_path}")

In [4]:
def sdf_diff_pointcloud(sdf1, sdf2, bounds, eps=0.01, step=2, top_q=0.98):
    """
    Visualize where two SDFs differ most, near the surface.
    eps: only keep points close to either surface
    step: subsample grid for speed
    top_q: keep only top quantile of differences (e.g., 0.98 = top 2%)
    """
    mn, mx = bounds
    sdf1 = np.asarray(sdf1)
    sdf2 = np.asarray(sdf2)

    # subsample grid
    sdf1s = sdf1[::step, ::step, ::step]
    sdf2s = sdf2[::step, ::step, ::step]

    diff = np.abs(sdf1s - sdf2s)

    # mask: only near either surface
    mask = (np.abs(sdf1s) < eps) | (np.abs(sdf2s) < eps)
    if not np.any(mask):
        raise ValueError("Mask is empty. Increase eps or check SDF sign/scale.")

    diff_m = diff[mask]

    # focus on largest differences
    thresh = np.quantile(diff_m, top_q)
    keep = mask & (diff >= thresh)

    # grid coords -> world coords
    nx, ny, nz = sdf1s.shape
    ijk = np.argwhere(keep)  # indices in subsampled grid
    frac = ijk / (np.array([nx-1, ny-1, nz-1], dtype=float))
    pts = mn + frac * (mx - mn)

    vals = diff[keep]
    vals = vals / (vals.max() + 1e-12)

    # color map: blue->red
    colors = np.stack([vals, np.zeros_like(vals), 1.0 - vals], axis=1)

    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(pts)
    pc.colors = o3d.utility.Vector3dVector(colors)

    return pc
 

### Ab hier in neues Notebook wegen regelmäßigen abstürzen
template_mesh = o3d.io.read_triangle_mesh("SDFD_sheep_mesh.ply")
template_mesh.compute_vertex_normals()

gazelle_mesh = o3d.io.read_triangle_mesh("SDFD_gazelle_mesh.ply")
gazelle_mesh.compute_vertex_normals()

pc = o3d.io.read_point_cloud("SDFD_Diff_Points.ply")
pc.paint_uniform_color([1.0, 0.0, 0.0])
#o3d.visualization.draw_geometries([pc], window_name="Top SDF differences near surface")

meshA = _copy_mesh(template_mesh)
meshA.paint_uniform_color([0.80, 0.70, 0.55])

meshB = _copy_mesh(gazelle_mesh)
meshB.paint_uniform_color([0.45, 0.60, 0.75])

# (optional) make sure normals exist for better shading
meshA.compute_vertex_normals()
meshB.compute_vertex_normals()

vis = o3d.visualization.Visualizer()
vis.create_window(window_name="Bones + SDF difference", width=1280, height=900)

vis.add_geometry(meshA)
vis.add_geometry(meshB)

if pc is not None and len(pc.points) > 0:
    vis.add_geometry(pc)

opt = vis.get_render_option()
opt.point_size = 12.0
opt.mesh_show_back_face = True

# CRITICAL: initialize view/camera
vis.poll_events()
vis.update_renderer()
vis.reset_view_point(True)

vis.run()
vis.destroy_window()

generate_gif([meshA, meshB, pc], [[0.80, 0.70, 0.55], [0.45, 0.60, 0.75], [1.0, 0.0, 0.0]], "SDFD-CompareTwoShapes.gif")

Saved rotating GIF as: SDFD-CompareTwoShapes.gif
